In [1]:
import os
import cv2
import glob
import torch
import shutil
import time
import copy
import itertools
import torch.nn as nn
import torch.optim as optim
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from pathlib import Path
from torch.nn import functional as F
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image

In [2]:
positive_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_positive\\'
negative_path = 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_sampled_negative\\'

In [3]:
os.path.exists(negative_path)

True

In [4]:
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data\\working\\train')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/train/positive')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test/positive')

# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/train/negative')
# os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/test/negative')

In [5]:
positive_train_len = int(np.floor(len(os.listdir(positive_path))*0.8))
positive_len = len(os.listdir(positive_path))

negative_train_len = int(np.floor(len(os.listdir(negative_path))*0.8))
negative_len = len(os.listdir(negative_path))

In [6]:
for trainimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/positive')
    
for trainimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len):
    shutil.copy(trainimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/train/negative')

for testimg in itertools.islice(glob.iglob(os.path.join(positive_path, '*.png')), positive_train_len, positive_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/positive')

for testimg in itertools.islice(glob.iglob(os.path.join(negative_path, '*.png')), negative_train_len, negative_len):
    shutil.copy(testimg, 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\equalized_data_model/working/test/negative')

In [7]:
normalizer = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        normalizer
    ]),
    
    'validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalizer
    ])
}

In [8]:
data_images = {
    'train': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/train', data_transforms['train']),
    'validation': datasets.ImageFolder('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data/equalized_data_model/working/test', data_transforms['validation'])
}

In [9]:
dataloaders = {
    'train': torch.utils.data.DataLoader(data_images['train'], batch_size=8, shuffle=True, num_workers=0),
    'validation': torch.utils.data.DataLoader(data_images['validation'], batch_size=8,shuffle=True,num_workers=0)
}

In [10]:
device_id = 0
torch.cuda.device_count()
torch.cuda.get_device_name(device_id)
# print(device_id)

'NVIDIA GeForce GTX 1660 Ti'

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
model.aux_logits=False
if torch.cuda.is_available():
    model.cuda()

Using cache found in C:\Users\arnav/.cache\torch\hub\pytorch_vision_v0.10.0


In [12]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
# for param in model.parameters():
#     param.requires_grad = False

In [14]:
model.fc = nn.Sequential(
    nn.Linear(2048, 64),
    nn.ReLU(inplace=True),
    nn.Linear(64, 3)
).to(device)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [16]:
# def trained_model(model, criterion, optimizer, epochs):
#     best_acc = 0.0
#     best_precision = 0.0
#     best_recall = 0.0
#     best_f1 = 0.0

#     for epoch in range(epochs):
        
#         print('Epoch:', str(epoch+1) + '/' + str(epochs))
#         print('-'*10)
        
#         for phase in ['train', 'validation']:
#             if phase == 'train':
#                 model.train() #this trains the model
#             else:
#                 model.eval() #this evaluates the model

#             running_loss, running_corrects = 0.0, 0 

#             iteration_count=0
#             total_precision=0
#             total_recall=0
#             total_f1=0
#             for inputs, labels in dataloaders[phase]:
#                 inputs = inputs.to(device) #convert inputs to cpu or cuda
#                 labels = labels.to(device) #convert labels to cpu or cuda

                
#                 # inputs = inputs.logits
#                 with torch.no_grad():
#                     outputs = model(inputs) #outputs is inputs being fed to the model
#                 # outputs = outputs.logits
#                 loss = criterion(outputs, labels) #outputs are fed into the model

#                 if phase == 'train':
#                     # optimizer.zero_grad() #sets gradients to zero
#                     loss.requires_grad=True
#                     loss.backward() #computes sum of gradients
#                     optimizer.step() #preforms an optimization step

#                 _, preds = torch.max(outputs, 1) #max elements of outputs with output dimension of one
#                 running_loss += loss.item() * inputs.size(0) #loss multiplied by the first dimension of inputs
#                 running_corrects += torch.sum(preds == labels.data) #sum of all the correct predictions

#                 iteration_count+=1
#                 precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
#                 total_precision+=precision_it
#                 total_recall+=recall_it
#                 total_f1+=f1_it

#             epoch_loss = running_loss / len(data_images[phase]) #this is the epoch loss
#             epoch_accuracy = running_corrects.double() / len(data_images[phase]) #this is the epoch accuracy
#             epoch_precision=total_precision/iteration_count
#             epoch_recall=total_recall/iteration_count
#             epoch_f1=total_f1/iteration_count

#             if phase == 'validation' and epoch_accuracy > best_acc:
#                 best_acc = epoch_accuracy
#                 best_precision = epoch_precision
#                 best_recall = epoch_recall
#                 best_f1 = epoch_f1

#             print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
#             phase, epoch_loss, epoch_accuracy, epoch_precision, epoch_recall, epoch_f1))
#             # print(phase, ' loss:', epoch_loss, 'epoch_accuracy:', epoch_accuracy)


#     print('Best val Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))
#     return model

In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    flag = 0

    for epoch in range(num_epochs):
        if flag == 1:
            break
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            iteration_count=0
            total_precision=0
            total_recall=0
            total_f1=0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                iteration_count+=1
                precision_it= precision_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                recall_it= recall_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                f1_it= f1_score(labels.cpu().numpy(),preds.cpu().numpy(), average='weighted', zero_division=0)
                total_precision+=precision_it
                total_recall+=recall_it
                total_f1+=f1_it

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(data_images[phase])
            epoch_acc = running_corrects.double() / len(data_images[phase])
            epoch_precision=total_precision/iteration_count
            epoch_recall=total_recall/iteration_count
            epoch_f1=total_f1/iteration_count

            print('{} Loss: {:.3f} Acc: {:.3f} Prec: {:.3f} Recall: {:.3f} F1 {:.3f}'.format(
                phase, epoch_loss, epoch_acc, epoch_precision, epoch_recall, epoch_f1))

            # deep copy the model and implement early stopping to prevent overfitting
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_precision = epoch_precision
                best_recall = epoch_recall
                best_f1 = epoch_f1
                es = 0
                best_model_wts = copy.deepcopy(model.state_dict())
            elif phase == 'validation':
                es += 1
                print("EarlyStopping counter {} of 3".format(es))

                if es > 300:
                    print('Early stopping with best_acc: {:4f} and val_acc for this epoch: {:.4f}'.format(best_acc,epoch_acc))
                    flag = 1
                

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.3f}, Best Precision: {:.3f}, Best Recall: {:.3f}, Best F1: {:.3f}'.format(best_acc, best_precision, best_recall, best_f1))


    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [18]:
optimizer_ft = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [19]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  542254 KB |  542254 KB |  542254 KB |       0 B  |\n|       from large pool |  540672 KB |  540672 KB |  540672 KB |       0 B  |\n|       from small pool |    1582 KB |    1582 KB |    1582 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  542254 KB |  542254 KB |  542254 KB |       0 B  |\n|       from large pool |  540672 KB |  540672 KB |

In [20]:
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=30)

Epoch 1/30
----------


C:\Users\arnav\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 0.887 Acc: 0.534 Prec: 0.524 Recall: 0.534 F1 0.481
validation Loss: 0.705 Acc: 0.400 Prec: 0.192 Recall: 0.402 F1 0.253

Epoch 2/30
----------
train Loss: 0.710 Acc: 0.509 Prec: 0.472 Recall: 0.509 F1 0.439
validation Loss: 0.717 Acc: 0.400 Prec: 0.182 Recall: 0.399 F1 0.245
EarlyStopping counter 1 of 3

Epoch 3/30
----------
train Loss: 0.704 Acc: 0.518 Prec: 0.458 Recall: 0.518 F1 0.449
validation Loss: 0.699 Acc: 0.400 Prec: 0.191 Recall: 0.400 F1 0.251
EarlyStopping counter 2 of 3

Epoch 4/30
----------
train Loss: 0.701 Acc: 0.502 Prec: 0.473 Recall: 0.502 F1 0.437
validation Loss: 0.679 Acc: 0.600 Prec: 0.400 Recall: 0.602 F1 0.473

Epoch 5/30
----------
train Loss: 0.699 Acc: 0.510 Prec: 0.469 Recall: 0.510 F1 0.442
validation Loss: 0.682 Acc: 0.600 Prec: 0.391 Recall: 0.599 F1 0.465
EarlyStopping counter 1 of 3

Epoch 6/30
----------
train Loss: 0.702 Acc: 0.497 Prec: 0.441 Recall: 0.497 F1 0.418
validation Loss: 0.684 Acc: 0.600 Prec: 0.386 Recall: 0.599 F1 0.463


In [21]:
os.mkdir('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/models')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\inceptionv3_model_data/working/models'

In [22]:
torch.save(model.state_dict(), 'C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\vgg16_model_weights/working/models/weights.h5') #save the model's weights
model.load_state_dict(torch.load('C:\\Users\\arnav\\Documents\\Arnav\\Research-IITR\\CovidX_Github_dataset\\data\\vgg16_model_weights/working/models/weights.h5')) #load the model's weights

<All keys matched successfully>